In [1]:
#file locations
filename = '../../../Applications/Assessment/loans.csv'
loan_profile = '../loan_report.html'

#Email settings
port = 2525 
smtp_server = "smtp.mailtrap.io"
login = "1a2b3c4d5e6f7g" # paste your login generated by Mailtrap
password = "1a2b3c4d5e6f7g" # paste your password generated by Mailtrap
subject = "ETL Pipeline Results"
sender_email = "sender@mailtrap.io"
receiver_email = "new@mailtrap.io"


#Database connections
dw_string = "host='localhost' dbname='dw1' user='dw' password='dwhouse'"

In [6]:
########## ETL pipeline
# ETL tools
import psycopg2
import pygrametl
from pygrametl.datasources import SQLSource, CSVSource, PandasSource
from pygrametl.tables import Dimension, FactTable, CachedDimension, BulkFactTable

# stats tools
import pandas as pd
import pandas_profiling

#email tools
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#OS tools
import os

# Ingest CSV input data file and modify data types
filename = os.path.abspath(filename)
names = ['End of Period','loannumber','region','countrycode','country','Borrower','Guarantor Country Code','Guarantor','Loan Type','Loan Status','Interest Rate','Currency of Commitment','projectidsrc','projectname' ,'orig_principal_amt','cancelled_amt','undisbursed_amt','disbursed_amt','repaid_to_ibrd','Due to IBRD','Exchange Adjustment','Borrower\'s Obligation','Sold 3rd Party','Repaid 3rd Party','Due 3rd Party','Loans Held','First Repayment Date','Last Repayment Date','Agreement Signing Date','Board Approval Date','Effective Date','Closed Date','Last Disbursement Date']
data = pd.read_csv(filename, names=names, skiprows=1)
data['projectidsrc'] = data['projectidsrc'].astype(str)
data['countrycode'] = data['countrycode'].astype(str).str.upper()
data['region'] = data['region'].astype(str).str.upper()
# df['End of Period'] = pd.to_numeric(df['End of Period'])
# df['Due 3rd Party'] = pd.to_datetime(df['Due 3rd Party']) 

ModuleNotFoundError: No module named 'psycopg2'

In [4]:
############Configs
'''
port = 2525 
smtp_server = "smtp.mailtrap.io"
login = "1a2b3c4d5e6f7g" # paste your login generated by Mailtrap
password = "1a2b3c4d5e6f7g" # paste your password generated by Mailtrap

subject = "ETL Pipeline Results"
sender_email = "sender@mailtrap.io"
receiver_email = "new@mailtrap.io"
loan_profile = "../loan_report.html"
total = len(data.index)
'''
total = len(data.index)

NameError: name 'data' is not defined

In [ ]:
##########Explore data --Commented out for brevity
'''
peek = data.head(20)
print(peek)
print(data.shape)
print(peek.shape)
types = data.dtypes
print(types)
types = data.dtypes
print(types)
pd.set_option( 'display.width' , 100)
pd.set_option( 'precision' , 3)
description = data.describe()
print(description)
class_counts = data.groupby( 'projectname' ).size()
print(class_counts)
data.isnull().sum()
print(data.head(3))
data.columns
'''

In [ ]:
##########Stats
# profile the data - Statistics
loan_profile = os.path.abspath(loan_profile)
profile = pandas_profiling.ProfileReport(data,minimal=True)
profile.to_file(loan_profile)

In [ ]:
##########Database
# datawarehouse connection
data.drop(columns=['Borrower','Guarantor Country Code','Guarantor','Loan Type','Loan Status','Interest Rate','Currency of Commitment','Due to IBRD','Exchange Adjustment','Borrower\'s Obligation','Sold 3rd Party','Repaid 3rd Party','Due 3rd Party','Loans Held','First Repayment Date','Last Repayment Date','Agreement Signing Date','Board Approval Date','Effective Date','Closed Date','Last Disbursement Date'])
dataPS = PandasSource(data)
dw_string = "host='localhost' dbname='dw1' user='dw' password='dwhouse'"
dw_pgconn = psycopg2.connect(dw_string)
dw_conn_wrapper = pygrametl.ConnectionWrapper(connection=dw_pgconn)


In [ ]:
##########Utility Methods
# bulk insert method
def bulkloader(name, attributes, fieldsep, rowsep, nullval, filehandle):
    cursor = dw_conn_wrapper.cursor()
    cursor.copy_from(file=filehandle, table=name, sep=fieldsep,null="nullval",
                     columns=attributes)
    
# split the timestamp into its parts
def split_timestamp(row):
    timestamp = row['End of Period']
    timestamp = timestamp[:10]
    timestamp_split = timestamp.split('-')
    row['year'] = timestamp_split[0]
    row['month'] = timestamp_split[1]
    row['day'] = timestamp_split[2]

# for sending email

def send_mail(sender_email, receiver_email, subject,  filename, total):
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject

    # Add body to email
    body = f"hi there, Please find attached stats from the monthly ETL pipeline,{total} records processed"
    message.attach(MIMEText(body, 'plain'))

    filename = filename


    # We assume that the file is in the directory where you run your Python script from
    with open(filename, 'rb') as attachment:
        # The content type "application/octet-stream" means that a MIME attachment is a binary file
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())

    # Encode to base64
    encoders.encode_base64(part)

    # Add header 
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to your message and convert it to string
    message.attach(part)
    text = message.as_string()

    # send your email
    with smtplib.SMTP("smtp.mailtrap.io", 2525) as server:
        server.login(login, password)
        server.sendmail(
            sender_email, receiver_email, text
        )
    print('Sent') 

In [ ]:
########## Dimension and Fact tables
# Dimension and Fact objects
loan_dimension = CachedDimension(
    name='loandim',
    key='loanid',
    attributes=['loannumber'],
    lookupatts=['loannumber'],
    prefill=True)

eop_dimension = CachedDimension(
    name='eopdim',
    key='eopid',
    attributes=['day', 'month', 'year'],
    lookupatts=['day','month','year'],
    prefill=True)

country_dimension = CachedDimension(
    name='countrydim',
    key='countryid',
    attributes=['countrycode', 'country'],
    lookupatts=['country'],
    prefill=True)

region_dimension = CachedDimension(
    name='regiondim',
    key='regionid',
    attributes=['region'],
    lookupatts=['region'],
    prefill=True)

project_dimension = CachedDimension(
    name='projectdim',
    key='projectid',
    attributes=['projectidsrc', 'projectname'],
    lookupatts=['projectidsrc'],
    prefill=True)

'''# for initial bulk load to improve performance

fact_table = BulkFactTable(
    name='loanfact',
    keyrefs=['loanid', 'eopid', 'countryid','regionid','projectid'],
    measures=['orig_principal_amt','cancelled_amt','undisbursed_amt','disbursed_amt','repaid_to_ibrd'],
    bulkloader=bulkloader,
    bulksize=1000000)
'''    


fact_table = FactTable(
        name='loanfact',
        keyrefs=['loanid', 'eopid', 'countryid','regionid','projectid'],
        measures=['orig_principal_amt','cancelled_amt','undisbursed_amt','disbursed_amt','repaid_to_ibrd'])


In [ ]:
########## Load Data
for row in dataPS:
    split_timestamp(row)
    row['loanid'] = loan_dimension.ensure(row)
    row['eopid'] = eop_dimension.ensure(row)
    row['countryid'] = country_dimension.ensure(row, namemapping={'countrycode':'countrycode'})
    row['regionid'] = region_dimension.ensure(row)
    row['projectid'] = project_dimension.ensure(row)
    #insert fact
    
    fact_table.ensure(row)
    #fact_table.insert(row)
    
dw_conn_wrapper.commit()
dw_conn_wrapper.close()

In [ ]:
##########Send email
# send_mail(sender_email, receiver_email, subject,  loan_profile, total)